In [48]:
import torch
import numpy as np
from scipy.stats import entropy, ks_2samp, moment, wasserstein_distance, energy_distance


# Computing metrics for different archtypes
def compute_divergences(A, B):
    """ Compute divergence metrics (Jensen Shannon, Kullback-Liebler,
    Wasserstein Distance, Energy Distance) between predicted distribution A
    and true distribution B """

    # Get number of samples, IQR statistics, range
    samples = A.shape[0]
    iqr = np.percentile(A, 75)-np.percentile(A, 25)
    r = np.max(A) - np.min(A)

    # Get PDFs of predicted distribution A, true distribution B
    B = get_pdf(B, iqr, r, samples)
    A = get_pdf(A, iqr, r, samples)
    
    # Mean
    m = (np.array(A)+np.array(B))/2

    # Compute metrics
    kl = entropy(pk=A, qk=B).sum()/A.shape[1]
    js = .5*(entropy(pk=A, qk=m)+entropy(pk=B, qk=m)).sum()/A.shape[1]
    wd = sum([wasserstein_distance(A[:,i], B[:,i]) for i in range(A.shape[1])])
    ed = sum([energy_distance(A[:,i], B[:,i]) for i in range(A.shape[1])])

    divergences = {"KL-Divergence": kl,
                    "Jensen-Shannon": js,
                    "Wasserstein-Distance": wd,
                    "Energy-Distance": ed}

    return divergences


def get_pdf(data, iqr, r, samples):
    """ Compute optimally binned probability distribution function  """
    x = []

    if iqr > 1e-5:
        bin_width = 2*iqr/np.cbrt(samples)
        bins = int(round(r/bin_width, 0))
    else:
        # MNIST (since it's really only supposed to be either 0 or 1 as output)
        # TODO: bin number
        bins = 2

    # Bin data
    for i in range(data.shape[1]):
        x.append(list(np.histogram(data[:, i], bins=bins, density=True)[0]))
    
    res = np.array(x).T
    res[res == 0] = .00001
    return res

In [49]:
def pdf_info(tensor, q1, q2):
    reshaped = tensor.view(-1)
    vals, _ = torch.sort(reshaped)
    lower_index = torch.tensor(len(vals)*(q1/100.), dtype=torch.long)
    upper_index = torch.tensor(len(vals)*(q2/100.), dtype=torch.long)
    iqr, r = vals[upper_index]-vals[lower_index], max(reshaped)-min(reshaped)
    return iqr, r

In [95]:
A = torch.rand(100, 384)
B = torch.rand(100, 384)

# Get IQR, range
iqr, r = pdf_info(A, 25, 75)

# Get number of samples, IQR statistics, range
samples = A.shape[0]

# Compute optimal number of bins
if iqr > 1e-5:
    bin_width = 2*iqr/np.cbrt(samples)
    bins = int(torch.round(r/bin_width))
else:
    # MNIST (since it's really only supposed to be either 0 or 1 as output)
    bins = 2


# Bin data
for i in range(A.shape[1]):
    split_sizes = list(torch.histc(A[:, i].unsqueeze(0), bins=bins))
    binned = torch.split(A[:, i], split_sizes)
    
# x = torch.stack(x, dim=0).t()
# x[x == 0.] = .0001
# res = np.array(x).T
# res[res == 0] = .00001

In [113]:
np.histogram(A[:, i], bins=bins, density=True)[0]

array([0.96379181, 1.16669534, 1.318873  , 0.6594365 , 0.96379181])

In [117]:
binned[0]

tensor([0.5120, 0.8679, 0.5436, 0.5864, 0.5975, 0.7310, 0.5374, 0.9440, 0.0122,
        0.5917, 0.7504, 0.3882, 0.5232, 0.7612, 0.9199, 0.3626, 0.1818, 0.1558,
        0.2462])

In [74]:
import torch.nn.functional as F

In [110]:
torch.stack([i.sum() for i in torch.split(A[:, i], split_sizes)])

tensor([10.2129, 11.5778, 13.4894,  5.9807,  7.4806])

In [112]:
binned

(tensor([0.5120, 0.8679, 0.5436, 0.5864, 0.5975, 0.7310, 0.5374, 0.9440, 0.0122,
         0.5917, 0.7504, 0.3882, 0.5232, 0.7612, 0.9199, 0.3626, 0.1818, 0.1558,
         0.2462]),
 tensor([0.0470, 0.6908, 0.2849, 0.0107, 0.5306, 0.3337, 0.1801, 0.5450, 0.8865,
         0.3464, 0.3411, 0.1018, 0.7874, 0.9964, 0.5675, 0.6523, 0.6392, 0.8868,
         0.2185, 0.4355, 0.9826, 0.8220, 0.2912]),
 tensor([0.3101, 0.0808, 0.4335, 0.1517, 0.8076, 0.1835, 0.7917, 0.7890, 0.9808,
         0.6403, 0.7438, 0.8658, 0.2525, 0.8125, 0.6645, 0.5700, 0.1434, 0.3205,
         0.5207, 0.8382, 0.3805, 0.3723, 0.4196, 0.4059, 0.5293, 0.4812]),
 tensor([0.5569, 0.4898, 0.8138, 0.9287, 0.3409, 0.9035, 0.5990, 0.1810, 0.2557,
         0.3364, 0.2599, 0.1176, 0.1975]),
 tensor([0.4249, 0.9302, 0.2209, 0.1877, 0.3374, 0.0939, 0.0505, 0.8372, 0.0981,
         0.2128, 0.2179, 0.7116, 0.4839, 0.0936, 0.2991, 0.8020, 0.5533, 0.4802,
         0.4455]))

In [21]:
np.histogram(A[:, i], bins=bins, density=True)[0]

array([1.16046955, 1.21092474, 0.90819356, 0.75682796, 1.00910395])

In [ ]:
torch.histc(, bins=bins)

In [25]:
sort = sorted(A[:, i])

In [28]:
(max(sort) - min(sort)) / bins

0.19819563627243042

In [43]:
np.histogram(A[:, 0])

(array([ 8, 11,  8,  9, 12,  7, 16, 11,  9,  9]),
 array([0.00870156, 0.10467611, 0.20065066, 0.2966252 , 0.39259975,
        0.4885743 , 0.58454884, 0.68052339, 0.77649794, 0.87247248,
        0.96844703]))

In [47]:
A[:, 0][8:19].sum()

5.8353577

In [33]:
[sum(np.histogram(A[:, i])[0])[0:i+1] for idx in range(len(np.histogram(A[:, i])[0]))]

IndexError: invalid index to scalar variable.

In [37]:
np.histogram(A[:, i])

IndexError: arrays used as indices must be of integer (or boolean) type

In [38]:
for i in np.histogram(A[:, i]):
    print(i)

IndexError: arrays used as indices must be of integer (or boolean) type

In [356]:
torch.distributions.distribution.Distribution(A)

Distribution()

In [358]:
help(F.kl_div)

Help on function kl_div in module torch.nn.functional:

kl_div(input, target, size_average=None, reduce=None, reduction='elementwise_mean')
    The `Kullback-Leibler divergence`_ Loss.
    
    See :class:`~torch.nn.KLDivLoss` for details.
    
    Args:
        input: Tensor of arbitrary shape
        target: Tensor of the same shape as input
        size_average (bool, optional): Deprecated (see :attr:`reduction`). By default,
            the losses are averaged over each loss element in the batch. Note that for
            some losses, there multiple elements per sample. If the field :attr:`size_average`
            is set to ``False``, the losses are instead summed for each minibatch. Ignored
            when reduce is ``False``. Default: ``True``
        reduce (bool, optional): Deprecated (see :attr:`reduction`). By default, the
            losses are averaged or summed over observations for each minibatch depending
            on :attr:`size_average`. When :attr:`reduce` is ``Fa

In [360]:
F.kl_div(F.log_softmax(A, dim=0), F.softmax(B, dim=0))

tensor(0.0001)